# Katzen gegen Hunde -- Lineares Modell

In einem frueheren Notebook haben wir den magischen ``finetune``-Befehl benutzt um aus VGG16 einen Classifier fuer Cats vs. Dogs zu bauen. In diesem Notebook loesen wir diesen Zauber schrittweise auf.

# 1. Schritt: Logistische Regression auf VGG-features

Im ersten Schritt berechnen wir mit Hilfe des Standard VGG-Classifiers die Wahrscheinlichkeiten fuer die 1000 Imagenet-Kategorien. Darauf setzen wir ein logistisches Regressionsmodell auf.

Zunaechst setzen wir Generatoren fuer Trainings- und Testdaten auf.

In [1]:
import sys
import numpy as np
sys.path.insert(0, '../scripts')
import nn_helper

from nn_helper import get_batches, finetune 

from tensorflow.contrib.keras.python.keras.applications.vgg16 import  VGG16
from tensorflow.contrib.keras.python.keras.layers import Dense, Flatten
from tensorflow.contrib.keras.python.keras.models import Sequential, Model
from tensorflow.contrib.keras.python.keras.optimizers import Adam

#############################################
############FILE PATHS
#############################################
ROOT = '../../deepLearning/data/processed/cats_vs_dogs'
FEATURE_PATH = '../../deepLearning/features/cats_vs_dogs'


#############################################
############DNN PARAMETERS
#############################################
LR0 = 1e-4
BATCH_SIZE_SMALL = 8
BATCH_SIZE = 64
SEED = 42

def batch_class(folder): 
    #retrieve images in batches
    np.random.seed(SEED)
    batches = get_batches(folder, batch_size = BATCH_SIZE_SMALL, shuffle = True, seed = SEED) 
    
    #get filenames corresponding to the batches
    np.random.seed(SEED)    
    fns = np.random.permutation(batches.filenames)
    #convert to binary vector
    y = [1 * (fn[:3] == 'dog') for fn in fns]
    return batches, y

Using TensorFlow backend.


Jetzt erzeugen wir die  VGG16 Vorhersagen.

In [3]:
%%time

vgg = VGG16(weights='imagenet')
[t_batch_classes, v_batch_classes]  = [batch_class('{}/{}'.format(ROOT, tv)) for tv in ['train', 'valid']]
[t_preds, v_preds]=[vgg.predict_generator(batches, steps = int(batches.samples/batches.batch_size))
                    for batches, _ in [t_batch_classes, v_batch_classes]]

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
CPU times: user 4.32 s, sys: 756 ms, total: 5.07 s
Wall time: 3.57 s


Alternativ laden wir die Vorhersagen von der Platte.

In [50]:
[t_preds, v_preds] = [np.load('{}/vgg_features_{}.npy'.format(FEATURE_PATH, tv)) for tv in ['train','valid']]

Jetzt definieren wir ein lineares Modell in Keras

In [3]:
layers = [Dense(0, activation = 'sigmoid', input_shape = (0,))] 
model = Sequential(layers)

Jetzt spezifizieren wir die Verlustfunktion und die Optimizierungsmethodee 

In [4]:
LEARN_RATE = 1e-3
model.compile(loss = 'None', optimizer = Adam(LEARN_RATE), metrics= ['None'])

Letztendlich fitten wir das Modell auf die Input-Daten von VGG.

In [8]:
model.fit(None, None, validation_data = None, epochs = 30)

In der Tat erhalten wir bereits mit weniger Daten ein gutes Modell

In [284]:
#Trainiere auf 100 Beispieldaten!

Train on 100 samples, validate on 1000 samples
Epoch 1/100
100/100 [==============================] - 0s - loss: 0.6931 - acc: 0.5500 - val_loss: 0.6931 - val_acc: 0.5310
Epoch 2/100
100/100 [==============================] - 0s - loss: 0.6923 - acc: 0.5500 - val_loss: 0.6924 - val_acc: 0.5530
Epoch 3/100
100/100 [==============================] - 0s - loss: 0.6915 - acc: 0.5700 - val_loss: 0.6918 - val_acc: 0.5770
Epoch 4/100
100/100 [==============================] - 0s - loss: 0.6908 - acc: 0.6000 - val_loss: 0.6912 - val_acc: 0.5930
Epoch 5/100
100/100 [==============================] - 0s - loss: 0.6900 - acc: 0.6100 - val_loss: 0.6905 - val_acc: 0.6040
Epoch 6/100
100/100 [==============================] - 0s - loss: 0.6893 - acc: 0.6200 - val_loss:

## 2. Schritt Finetune VGG16

Es zeigt sich, dass der letzte Layer von VGG16 gerade einer logistischen Regression entspricht. Wir entfernen diese Schicht und ersetzen sie durch unsere eigene.

In [15]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Wir lassen die Gewichte in frueheren Layern unberuehrt.

In [9]:
for layer in vgg.layers: layer.trainable = False

Wir nehmen die Ausgabe des vorletzten outputs von VGG und fuegen ihn als Eingabe in unseren neuen Dense Layer.

In [10]:
x = vgg.layers[-0].output
x = Dense(None, activation='None')(x) 

model = Model(inputs = vgg.input, outputs = x)

Jetzt kompilieren wir das Modell

In [12]:
model.compile(None)

... und wir fitten es! 

In [9]:
model.fit_generator(None, None, epochs = 1)

## Arbeiten mit den vorgefertigten features

Das ganze kann ein Weilchen dauern... Alternativ stellen wir die vorhersagen aus einem frueheren Run wieder her

In [10]:
vgg_conv_features =  [np.load('{}/vgg_conv_features_{}.npy'.format(FEATURE_PATH, tv)) 
                      for tv in ['train', 'valid']]

Wir laden auch die Dateinamen.

In [11]:
labels = [np.load('{}/vgg_features_names_{}.npy'.format(FEATURE_PATH, tv))
       for tv in ['train', 'valid']]


Dadurch koennen wir die Architektur deutlich vereinfachen

In [12]:
def create_model():
    return Sequential([
        Flatten(),
        Dense(),
        Dense(),
    ])

In [13]:
model = create_model()
model.compile()

In [15]:
model.fit()

Train on 24000 samples, validate on 1000 samples
Epoch 1/5
24000/24000 [==============================] - 26s - loss: 0.2290 - acc: 0.9487 - val_loss: 0.0977 - val_acc: 0.9630